In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('../../')))
import logging
# Logging setup
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

module_logger = logging.getLogger()
module_logger.addHandler(handler)
module_logger.setLevel(logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.base import LanguageModel, Chain, Prompt, ProbabilisticComponent, Threads, Concept, ConceptRegistry, use_registry
import pandas as pd
import dtale

# tests

In [4]:
# pd.DataFrame(chain.memory)
# dtale.show(pd.DataFrame(lm3.memory))

# parsers
# list as output
# routers
# retriever


In [5]:
# add placeholders and import 

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), Prompt("tell me a joke about {subject}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google1"), Prompt("what do you think of this joke: {joke}?"), Concept('other')), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), Prompt("can you explain why this joke is funny: {joke}?"), Concept('subject'))])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), Concept('think')), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), Concept('lala'))])

threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {else}?"), Concept('think')), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), Concept('kaka'))])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), Concept('joke'))

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

## Component

In [6]:
initial_concepts = [Concept(name='other', string_content='computer'), Concept(name='something', string_content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    component.run()

In [7]:
component.memory

['Response of OpenAI2: to (tell me a joke about computer.)']

## Chain

In [8]:
# Run the chain
initial_concepts = [Concept(name='something', string_content='something'), Concept(name='subject', string_content='computer')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    out = chain1.run()

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:180: UserWarning:

Overwrite Warning: 
{'choice': 'all',
 'inputted': True,
 'level': 0,
 'list_concepts': None,
 'list_content': None,
 'listify_func': <function listify_func at 0x7fe8b4afbb50>,
 'name': 'subject',
 'string_content': 'computer',
 'type': 'identity'}

 Is being overwritten by:
 {'choice': 'all',
 'inputted': False,
 'level': 2,
 'list_concepts': None,
 'list_content': None,
 'listify_func': <function listify_func at 0x7fe8b4afbb50>,
 'name': 'subject',
 'string_content': 'Response of Microsoft1: to (can you explain why this joke '
                   'is funny: Response of OpenAI1: to (tell me a joke about '
                   'computer.)?)',
 'type': 'identity'}



In [9]:
concept_registry

ConceptRegistry:
  - something:
    Name: something
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: something
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - joke:
    Name: joke
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of OpenAI1: to (tell me a joke about computer.)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 0
  - other:
    Name: other
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 1
  - subject:
    Name: subject
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
   

## Chain of chains

In [10]:
chain3 = Chain([chain1, chain2])

In [11]:

initial_concepts = [Concept(name='subject', string_content='computer'), Concept(name='else', string_content='else')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain3.run()
concept_registry

ConceptRegistry:
  - subject:
    Name: subject
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: computer
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - else:
    Name: else
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: else
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - joke:
    Name: joke
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of OpenAI2: to (tell me a joke about computer.)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 0
  - think:
    Name: think
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about com

## Threads

In [12]:
initial_concepts = [Concept(name='subject', string_content='computer'), Concept(name='lala', string_content='something'), 
                        Concept(name='think', string_content='something'), Concept(name='else', string_content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    threads1._run()


/Users/fuad/Documents/projects/lexflow/src/base/executable.py:180: UserWarning:

Overwrite Warning: 
{'choice': 'all',
 'inputted': True,
 'level': 0,
 'list_concepts': None,
 'list_content': None,
 'listify_func': <function listify_func at 0x7fe8b4afbb50>,
 'name': 'think',
 'string_content': 'something',
 'type': 'identity'}

 Is being overwritten by:
 {'choice': 'all',
 'inputted': False,
 'level': 1,
 'list_concepts': None,
 'list_content': None,
 'listify_func': <function listify_func at 0x7fe8b4afbb50>,
 'name': 'think',
 'string_content': 'Response of Google2: to (what do you think of this joke: '
                   'something?)',
 'type': 'identity'}



In [13]:
concept_registry

ConceptRegistry:
  - subject:
    Name: subject
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: computer
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - lala:
    Name: lala
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: something
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - else:
    Name: else
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: something
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - think:
    Name: think
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of Google2: to (what do you think of this joke: something?)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 1


# More complex

In [14]:
# add placeholders and import 
threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {something}?"), Concept('think')), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), Concept('kaka'))])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), Concept('think')), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), Concept('lala')),
                threads1
               ])

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), 
                          Prompt("tell me a joke about {subject}."), Concept('haha')), 
                ProbabilisticComponent(LanguageModel("Google1"), 
                          Prompt("what do you think of this joke: {haha}?"), Concept('other')), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {haha}?"), Concept('subject')),
                chain2
               ])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), Concept('joke'))



In [15]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

chain3 = Chain([chain1, chain2, threads1])
initial_concepts = [Concept(name='subject', string_content='computer'), Concept(name='something', string_content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:180: UserWarning:

Overwrite Warning: 
{'choice': 'all',
 'inputted': False,
 'level': 3,
 'list_concepts': None,
 'list_content': None,
 'listify_func': <function listify_func at 0x7fe8b4afbb50>,
 'name': 'joke',
 'string_content': 'Response of OpenAI2: to (tell me a joke about Response of '
                   'Microsoft1: to (can you explain why this joke is funny: '
                   'Response of OpenAI1: to (tell me a joke about '
                   'computer.)?).)',
 'type': 'identity'}

 Is being overwritten by:
 {'choice': 'all',
 'inputted': False,
 'level': 7,
 'list_concepts': None,
 'list_content': None,
 'listify_func': <function listify_func at 0x7fe8b4afbb50>,
 'name': 'joke',
 'string_content': 'Response of OpenAI2: to (tell me a joke about Response of '
                   'Microsoft2: to (can you explain why this joke is funny: '
                   'Response of Google2: to (what do you think of this joke: '


In [16]:
concept_registry

ConceptRegistry:
  - something:
    Name: something
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: something
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - haha:
    Name: haha
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of OpenAI1: to (tell me a joke about computer.)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 0
  - other:
    Name: other
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 1
  - subject:
    Name: subject
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
   

In [17]:
# object names should be unique otherwise they will be overwritten 

# Listify

In [18]:
# add placeholders and import 
threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {something}?"), Concept('think')), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), Concept('kaka'))])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), Concept('joke')), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), Concept('think')), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), Concept('lala')),
                threads1
               ])

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), 
                          Prompt("tell me a joke about {subject}."), Concept('haha')), 
                ProbabilisticComponent(LanguageModel("Google1"), 
                          Prompt("what do you think of this joke: {haha}?"), Concept('other')), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {haha}?"), Concept('subject')),
                chain2
               ])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), Concept('joke'))

chain3 = Chain([chain1, chain2, threads1])


In [22]:
comp1 =ProbabilisticComponent.create(**{
        'model': LanguageModel("OpenAI1"),
        'prompt': Prompt("tell me a joke about {subject}."),
        'output_concept': Concept('joke')
    })

chain1 = Chain([comp1])


In [25]:
configs = [
    {
        'model': LanguageModel("OpenAI1"),
        'prompt': Prompt("tell me a joke about {subject}."),
        'output_concept': Concept('joke')
    },
    {
        'model': LanguageModel("OpenAI2"),
        'prompt': Prompt("what do you think of this joke: {joke}?"),
        'output_concept': Concept('think')
    }
    # ... add more configs as needed
]
comp1 = ProbabilisticComponent.create_iterable(configs)
chain1 = Chain(comp1)

In [26]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders


initial_concepts = [Concept(name='subject', string_content='computer'), Concept(name='something', string_content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

In [27]:
concept_registry

ConceptRegistry:
  - subject:
    Name: subject
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: computer
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - something:
    Name: something
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: something
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - joke:
    Name: joke
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of OpenAI1: to (tell me a joke about computer.)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 0
  - think:
    Name: think
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe8b4afbb50>
    String_content: Response of OpenAI2: to (what do you think of this joke: Response of OpenAI1: to (tell me a